In [ ]:
from lxml import etree 
import time 
import logging
import os
import sqlite3
import pathlib
import glob
import gzip 
import csv 

import pandas as pd
import xmltodict

DOSSIER_PARENT = "."
DOSSIER_SOURCE = "./todo/"
DOSSIER_SORTIE = "./done/"
BDD = 'bdd_actes_budgetaires.db'
FICHIER_CSV = 'donnees_actes_budgetaires.csv'
FICHIERS_TO_DO= glob.glob(os.path.join(DOSSIER_SOURCE, "*.gz"))

In [ ]:
def timing_decorator_x10x(func):
 def wrapper(*args, **kwargs):
  total_time = 0
  num_runs = 10
  for _ in range(num_runs):
   start_time = time.time()
   func(*args, **kwargs)
   end_time = time.time()
   total_time += end_time - start_time
   average_time = total_time / num_runs
   print(f"Temps moyen d'exécution de {func.__name__}: {average_time:.6f} secondes")
 return wrapper

def timing_decorator(func):
 def wrapper(*args, **kwargs):
  start_time = time.time()
  result = func(*args, **kwargs)
  end_time = time.time()
  execution_time = end_time - start_time
  print(f"{func.__name__} a pris {execution_time:.4f} secondes pour s'exécuter.")
  return result
 return wrapper

In [ ]:
@timing_decorator
def parsing_fichier(chemin) : 
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def _extract_lignes_budget(data_dict: dict) -> pd.DataFrame : 
 "Sépare les sous clefs lignes budget, sans nettoyage"
 ligne_budget = data_dict['DocumentBudgetaire']['Budget']['LigneBudget']
 df_ligne_budget = pd.DataFrame(ligne_budget)
 return df_ligne_budget

def _extract_id(fichier) -> dict :
 "Extrait l'ID du nom du fichier, en fait un dict qui sera assemblé aux métadonnées"
 val_IdFichier = fichier.split("-")[-1].split('.')[0]
 IdFichier = {"IdFichier" : val_IdFichier}
 return IdFichier

def _extract_metadonnees(data_dict : dict, IdFichier) -> pd.DataFrame : 
 "Sépare les sous clefs de métadonnées, sans nettoyage"
 entete_dict = data_dict['DocumentBudgetaire']['EnTeteDocBudgetaire']
 IdFichier_dict = _extract_id(IdFichier)
 metadonnees_dict = {**IdFichier_dict, **entete_dict}
 df_metadonnees = pd.DataFrame(metadonnees_dict)
 return df_metadonnees

def extraction_donnees(data_dict : dict, fichier) -> pd.DataFrame :
 df_lignes_budget = _extract_lignes_budget(data_dict)
 df_metadonnees = _extract_metadonnees(data_dict, fichier)
 return df_lignes_budget, df_metadonnees

def nettoyage_lambda(df : pd.DataFrame) -> pd.DataFrame : 
 "Nettoie les données pour se débarasser des @V"
 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in df.columns : 
  df[col] = df[col].apply(nettoyage)
 return df 

def assemblage_unitaire(df_lignes_budget : pd.DataFrame, df_metadonnees : pd.DataFrame) -> pd.DataFrame: 
 """ Assemble les dataFrame contenant les metadonnees,
 la version schema et les lignes budgetaires """
 colonnes_a_conserver = ["IdFichier", "DteStr", "LibelleColl",
                         "IdColl","Nature","LibCpte",
                         "Fonction","Operation",
                         "ContNat","ArtSpe",
                         "ContFon", "ContOp",
                         "CodRD","MtBudgPrec",
                         "MtRARPrec","MtPropNouv",
                         "MtPrev","CredOuv",
                         "MtReal","MtRAR3112",
                         "OpBudg","TypOpBudg",
                         "OpeCpteTiers"
                        ]
 df_metadonnees = pd.concat([df_metadonnees] * len(df_lignes_budget), ignore_index=True)
 df_concat = pd.concat([df_metadonnees, df_lignes_budget], axis = 1)
 df_fichier = pd.DataFrame(columns=colonnes_a_conserver)

 for i in df_concat.columns : 
    if i in colonnes_a_conserver :
        df_fichier[i] = df_concat[i]

 df_fichier = df_fichier.dropna(axis = 1, how = 'all')
 return df_fichier

def insertion_csv(df_final : pd.DataFrame) :
 ''' S'ajoute à un csv déjà existant, permet de ne pas créer un csv à chaque fois '''
 with open(FICHIER_CSV, 'a', newline='') as fichier_csv : 
  ajout = csv.writer(fichier_csv)
  for _, ligne in df_final.iterrows():
   ajout.writerow(ligne)

def insertion_bdd(df_final: pd.DataFrame):
 """ insert dans une bdd les données maintenant transformées et en sort un csv à jour """
 chemin_bdd = os.path.join(DOSSIER_PARENT, BDD)
 conn = sqlite3.connect(chemin_bdd)
 df_final.to_sql('actes_budgetaire', conn,
                    if_exists='append', index=False)

def deplacement_fichier(fichier_a_deplacer, dossier_destination):
 """ Déplace le fichier du dossier source au dossier fini"""
 chemin_source = pathlib.Path(fichier_a_deplacer)
 chemin_destination = pathlib.Path(dossier_destination) / chemin_source.name
 chemin_source.rename(chemin_destination)


In [ ]:
def isolement_id(fichier) : 
 "Extrait l'id du nom du fichier pour la liste comprehension de securité"
 val_IdFichier = fichier.split("-")[-1].split('.')[0]
 return val_IdFichier

def recherche_id_dans_bdd():
    conn = sqlite3.connect(BDD)
    cursor = conn.cursor()
    cursor.execute(''' SELECT DISTINCT IdFichier FROM actes_budgetaire ''')
    ligne_sql_int = [int(x[0]) for x in cursor.fetchall()]
    conn.close()
    return ligne_sql_int

In [ ]:
def clean_csv() : 
 with open(FICHIER_CSV, 'w', newline='') as csvfile:
  header = "IdFichier,DteStr,LibelleColl,IdColl,Nature,LibCpte,Fonction,Operation,ContNat,ArtSpe,ContFon,ContOp,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,TypOpBudg,OpeCpteTiers\n"
  csvfile.write(header)

def clean_bdd() :  
 conn = sqlite3.connect(BDD)
 cursor = conn.cursor()
 cursor.execute('''DELETE FROM actes_budgetaire ''')
 conn.commit()
 conn.close()

In [ ]:
def main_methode_unitaire():
 """ Traitement global, extrait et transforme les fichiers XML dans DOSSIER_SOURCE
    pour les insérer dans une bdd et en faire un csv"""
 fichiers_safe = [fichier for fichier in FICHIERS_TO_DO if int(isolement_id(fichier)) not in recherche_id_dans_bdd()]
 for fichier in fichiers_safe : 
  fichier_parse = parsing_fichier(fichier)
  df_lignes_budget, df_metadonnees = extraction_donnees(fichier_parse, fichier)
  df_fichier = assemblage_unitaire(df_lignes_budget, df_metadonnees)
  df_fichier_propre = nettoyage_lambda(df_fichier)
  insertion_bdd(df_fichier_propre)
  insertion_csv(df_fichier_propre)

def main_solo(fichier): 
  fichier_parse = parsing_fichier(fichier)
  df_lignes_budget, df_metadonnees = extraction_donnees(fichier_parse, fichier)
  df_fichier = assemblage_unitaire(df_lignes_budget, df_metadonnees)
  df_fichier_propre = nettoyage_lambda(df_fichier)
  insertion_bdd(df_fichier_propre)
  insertion_csv(df_fichier_propre)
  return df_fichier_propre

In [ ]:
main_methode_unitaire()


In [ ]:
#clean_bdd()
#clean_csv()

In [ ]:
fichiers_safe = [fichier for fichier in FICHIERS_TO_DO if int(isolement_id(fichier)) not in recherche_id_dans_bdd()]


In [ ]:
main_methode_unitaire()
